In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from pandas import DataFrame

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [9]:
data = pd.read_csv('../input/train.csv')
data.head()

In [10]:
print("number of features:" + str(len(data.columns.tolist())))

#we will select some of these features that we think would influance 
#house prices. some of these features are categorical, ordinal or numerical
cat_f = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'Utilities', 
        'Neighborhood', 'BldgType', 'HouseStyle', 'Heating', 'LandContour', 'LotConfig',
        'LandSlope', 'Condition1', 'Condition2', 'RoofStyle', 'RoofMatl',
        'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'CentralAir', 'Electrical',
        'Functional', 'GarageType', 'GarageFinish', 'PavedDrive', 'SaleType', 'SaleCondition']
num_f = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearRemodAdd',
        'TotalBsmtSF', 'SalePrice']
ord_f = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 
         'GarageQual','GarageCond']

all_clms = cat_f + num_f + ord_f
data = data[all_clms]

#converting ord_f into numbers
qual = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po': 5}

#for missing values we take the mean value of quality which is 3
for col in ord_f:
    data[col] = data[col].apply(lambda x: 3 if x is np.nan else qual[x])

#for missing values of numerical columns we fill it with the mean also
for col in num_f:
    median = data[col].median()
    data[col].where(pd.notnull(data[col]), median, inplace=True)

for col in cat_f:
    data[col].where(pd.notnull(data[col]), 'not_a', inplace=True)
    
#apply get dummy method to convert the categorical features to numerical
data = pd.get_dummies(data)

In [11]:
data.head()

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

#splitting data to train and test and try to run GBTR
X_train, X_test, y_train, y_test = train_test_split(data.drop(['SalePrice'], axis=1),
                                                    data['SalePrice'],
                                                    test_size=0.30, random_state=45)

# fit estimator
est = GradientBoostingRegressor(n_estimators=100, max_depth=1, loss='ls')
est.fit(X_train, y_train)

# predict class labels
pred = est.predict(X_test)

# score on test data (accuracy)
acc = est.score(X_test, y_test)
print('ACC: %.4f' % acc)

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [4, 6],
              'min_samples_leaf': [3, 5, 9, 17],
              }

est = GradientBoostingRegressor(n_estimators=3000)
gs_cv = GridSearchCV(est, param_grid, n_jobs=4).fit(X_train, y_train)

# best hyperparameter setting
print(gs_cv.best_params_)
print(gs_cv.best_score_)

In [14]:
validation_set = pd.read_csv('../input/test.csv')

In [15]:
validation_set.head()

In [16]:
validation_set_id = validation_set['Id']

all_clms.remove('SalePrice')
validation_set = validation_set[all_clms]

#for missing values we take the mean value of quality which is 3
for col in ord_f:
    validation_set[col] = validation_set[col].apply(lambda x: 3 if x is np.nan else qual[x])

#for missing values of numerical columns we fill it with the mean also
num_f.remove('SalePrice')
for col in num_f:
    median = validation_set[col].median()
    validation_set[col].where(pd.notnull(validation_set[col]), median, inplace=True)

for col in cat_f:
    validation_set[col].where(pd.notnull(validation_set[col]), 'not_a', inplace=True)
    
#apply get dummy method to convert the categorical features to numerical
validation_set = pd.get_dummies(validation_set)

#the validation set might have extra values that turns into columns
#with git_dummy function or it might have less number of categorical values and won't produce
#all the columns
extra_clms   = list(set(validation_set.columns.tolist()) - set(X_train.columns.tolist()))
missing_clms = list(set(X_train.columns.tolist()) - set(validation_set.columns.tolist()))

#extending the validation set with the extra clms
for c in missing_clms:
    validation_set[c] = 0

#removing the columns from validation_set that doesnt exist in the model
for c in extra_clms:
    validation_set = validation_set.drop(c, axis=1)

In [18]:
prds = gs_cv.best_estimator_.predict(validation_set)
result = pd.DataFrame({'Id': validation_set_id, 'y': prds})
result.to_csv('predcs_new.csv', columns=['Id', 'y'], header=['Id', 'SalePrice'], index=False)